# Project 3: Getting Started 

This notebook is intended to help you get off to a flying start with the cars dataset. You don't have to use this notebook and you can discard any parts you do not like, they are purely intended as a help to get started. 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

# optimization
from scipy import optimize
import estimation as est
import clogit_post as clogit


import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

# Read in data

The dataset, `cars.csv`, contains cleaned and processed data. If you want to make changes, the notebook, `materialize.ipynb`, creates the data from the raw source datsets. 

In [2]:
cars = pd.read_csv('cars.csv')
lbl_vars = pd.read_csv('labels_variables.csv')
lbl_vals = pd.read_csv('labels_values.csv')

# convert from dataframe to dict
lbl_vals = {c: lbl_vals[c].dropna().to_dict() for c in lbl_vals.columns}

In [3]:
lbl_vars.set_index('variable', inplace=True)

## Overview of the dataset

In [4]:
lbl_vars.join(cars.mean(numeric_only=True).apply(lambda x: f'{x: .2f}').to_frame('Mean'))

,label,Mean
variable,,
ye,year (=first dimension of panel),84.50
ma,market (=second dimension of panel),3.00
co,model code (=third dimension of panel),207.50
zcode,alternative model code (predecessors and succe...,177.76
brd,brand code,16.79
type,name of brand and model,NaN
brand,name of brand,NaN
model,name of model,NaN
org,"origin code (demand side, country with which c...",2.72


# Set up for analysis

In [5]:
price_var = 'princ'

In [6]:
cars['logp'] = np.log(cars[price_var]) # Problem set hint

In [7]:
cars['logp'] = (cars[price_var])/np.log((cars['ngdp']/cars['pop'])) # Brownstone and Train

In [8]:
cars['logp'] = (cars[price_var])  # Brenkers and Verboven


In [9]:
np.mean(cars['logp'])

0.7557170818646749

In [10]:
# new variable: price elasticity heterogeneous for home-region 
cars['logp_x_home'] = cars[price_var] * cars['home']

### Dummy variables

For working with matrices, we want to have a column for each dummy variable. 

In [11]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar

x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])
print(len(x_vars_dummies))
# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

32


### `x_vars`: List of regressors to be used 

In [12]:
x_vars = ['logp', 'home', 'cy', 'hp', 'we', 'li'] + x_vars_dummies # <--- !!! choose your preferred variables here 
print(f'K = {len(x_vars)} variables selected.')

K = 38 variables selected.


In [13]:
print(x_vars_dummies)

['MCC', 'VW', 'alfa romeo', 'audi', 'citroen', 'daewoo', 'daf', 'fiat', 'ford', 'honda', 'hyundai', 'innocenti', 'lancia', 'mazda', 'mercedes', 'mitsubishi', 'nissan', 'opel', 'peugeot', 'renault', 'rover', 'saab', 'seat', 'skoda', 'suzuki', 'tal/hillman', 'tal/matra', 'tal/simca', 'tal/sunb', 'talbot', 'toyota', 'volvo']


In [14]:
K = len(x_vars)
N = cars.ma.nunique() * cars.ye.nunique()
J = 40 
x = cars[x_vars].values.reshape((N,J,K))
y = np.log(cars['s'].values.reshape((N,J)))

# standardize x
# x = ((x - x.mean(0).mean(0))/(x.std(0).std(0)))
print(x.shape)

(150, 40, 38)


### Understanding the sorting 

Just to be sure that we understand the relation between the pandas dataframe and the numpy 3d array, consider the following: 

In [15]:
# let's check that we get the same row from x as we can find in the original pandas dataframe
# we'll pick the first 5 "observations"
j = 1
k = 0 
x[:5, j, k] == cars.groupby(['ma','ye']).nth(j)[x_vars[k]].head(5).values

array([ True,  True,  True,  True,  True])

In [16]:
# ... and let's check it for the 5 first cars (in the first market)
k = 0
x[0, :5, k] == cars[x_vars[k]].head(5).values
# note that with i = 3 (4th element), x[i,t,k] gives ma=1 and ye=73 (first market, fourth year)
x[3, :5, k] == cars.query('(ma == 1) & (ye == 73)')[x_vars[k]].head(5).values

array([ True,  True,  True,  True,  True])

In [17]:
# and let's print out some rows along with some labels 
obs_labs = cars[['ma', 'ye', 'type', 's']].values.reshape(N,J,4) # notice that we are extracting the values from the dataframe in the same way as we did for x

i=3 # obs. index 3 is the first market in the fourth (3+1) year, i.e. 73
print(obs_labs[i,:5,:])

i = 130 # obs. index 130 is the 5th country (130/30>4) and the 11th year (130%30 = index 10)
print(obs_labs[i,:5,:])

[[1 73 'audi 80/90' 0.0198967806548532]
 [1 73 'audi 100/200' 0.0115738123314003]
 [1 73 'citroen 2 CV 6 - 2 CV 4' 0.020470221461224]
 [1 73 'citroen GSA/GSX' 0.0231960844492545]
 [1 73 'citroen dyane' 0.0232687741289353]]
[[5 80 'alfasud' 0.0061322294468038]
 [5 80 'citroen GSA' 0.0097859984028077]
 [5 80 'fiat 127' 0.0082314207084408]
 [5 80 'fiat 131F' 0.0099803206146036]
 [5 80 'ford fiesta' 0.0781217905939526]]


... and just checking that we can find those same columns in the pandas dataframe

In [18]:
cars.query('(ma == 5) & (ye == 80) & (type == "ford fiesta")').s

5204    0.078122
Name: s, dtype: float64

# OLS Example

Let's compute the OLS estimator just to test that we can do algebra with the arrays. 

***Note:*** This particular choice of $y$ and $x$ variables might not make sense, it is just to help you get started doing algebra on these arrays. 

In [19]:
Y = y.reshape(N*J,) # Make Y 1-dimensional 
X = np.hstack([x.reshape(N*J,K), np.ones((N*J,1))]).astype(np.float64) # append a constant term and ensure type = float

In [20]:
X.mean(axis=0)

array([7.55717082e-01, 3.16333333e-01, 1.33709042e+03, 5.01000167e+01,
       9.34488833e+02, 7.87377500e+00, 3.33333333e-04, 7.51666667e-02,
       2.26666667e-02, 3.75000000e-02, 7.65000000e-02, 1.00000000e-03,
       4.50000000e-03, 9.46666667e-02, 8.71666667e-02, 1.61666667e-02,
       1.33333333e-03, 2.50000000e-03, 2.08333333e-02, 1.96666667e-02,
       3.55000000e-02, 8.00000000e-03, 4.13333333e-02, 8.28333333e-02,
       7.33333333e-02, 1.09333333e-01, 4.46666667e-02, 1.16666667e-03,
       1.48333333e-02, 3.66666667e-03, 1.16666667e-03, 1.66666667e-03,
       3.33333333e-04, 1.16666667e-02, 1.66666667e-04, 2.08333333e-02,
       2.81666667e-02, 2.01666667e-02, 1.00000000e+00])

In [21]:
# compute the OLS estimator 
bet = np.linalg.inv(X.T @ X) @ X.T @ Y

# print
varnames = x_vars + ['const'] # we added the constant as the K+1'th column 
pd.DataFrame({'Estimate':bet}, index=varnames)

,Estimate
logp,-0.583040
home,0.965552
cy,-0.000295
hp,-0.008302
we,0.001100
li,-0.050792
MCC,-1.169253
VW,0.118337
alfa romeo,-0.487959
audi,-0.086136


# Towards logit 

In order to work with the logit model, you have to be able to compute the utility indices, which typically take the form of some inner product of an $x$-vector and a $\theta$ vector. This is illustrated for you below. Since `x` is `(N,J,K)` (i.e. `x[i,j,:]` gives the $K$-vector of regressors for the car `j` in market-period `i`), we just have to form the matrix product `x @ theta`, and Python will do the sum over the 3rd dimension of `x`. 

In [22]:
theta0 = np.zeros((K,))
v = (x @ theta0).astype(np.float64) # how to multiply a trial value with the matrix of regressors 
np.exp(v) / np.sum(np.exp(v), 1, keepdims=True) # choice probabilities 

array([[0.025, 0.025, 0.025, ..., 0.025, 0.025, 0.025],
       [0.025, 0.025, 0.025, ..., 0.025, 0.025, 0.025],
       [0.025, 0.025, 0.025, ..., 0.025, 0.025, 0.025],
       ...,
       [0.025, 0.025, 0.025, ..., 0.025, 0.025, 0.025],
       [0.025, 0.025, 0.025, ..., 0.025, 0.025, 0.025],
       [0.025, 0.025, 0.025, ..., 0.025, 0.025, 0.025]])

# Conditional logit estimation



In [23]:
y = cars['s'].values.reshape((N,J))


In [24]:
res = est.estimate(clogit.q, theta0,  y, x, cov_type = "Sandwich")
#print(res['log_like'])
tab = pd.DataFrame({v:res[v] for v in ['theta', 'se', 't']}, index= x_vars)
tab


c:\Users\vkt823\Introprog\My-repository\Assignments - Own attempt\Assignment 3\clogit_post.py:61: RuntimeWarning: divide by zero encountered in log
  temp = np.multiply(y, np.log(ccp))
c:\Users\vkt823\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
c:\Users\vkt823\Introprog\My-repository\Assignments - Own attempt\Assignment 3\clogit_post.py:61: RuntimeWarning: divide by zero encountered in log
  temp = np.multiply(y, np.log(ccp))


Optimization terminated successfully.
         Current function value: 3.467854
         Iterations: 265
         Function evaluations: 10617
         Gradient evaluations: 272


,theta,se,t
logp,-1.315485,0.109377,-12.027092
home,1.352751,0.032527,41.588692
cy,-0.000062,0.000074,-0.834994
hp,-0.005978,0.002036,-2.935605
we,0.000949,0.000147,6.449583
li,-0.036311,0.016428,-2.210257
MCC,-1.287829,0.147730,-8.717446
VW,0.129952,0.057968,2.241771
alfa romeo,-0.778207,0.072332,-10.758852
audi,-0.097308,0.068731,-1.415791


In [25]:
print(res['log_like'])

-553.3319181170893


# Price Elasticities

In [26]:
thetahat = res['theta']
# Calculate the original choice probabilities using the estimated parameters
ccp1 = clogit.choice_prob(thetahat, x)

E_own   = np.zeros((N, J))

# Due to log price variable being the first element of x_vars, we can use k_price = 0
k_price = 0 

for j in range(J):
    # A. copy 
    x2 = x.copy()
    
    # B. increase price just for car j 
    rel_change_x = 1e-3
    x2[:, j, k_price] *= (1.0+rel_change_x)
    
    # C. evaluate CCPs.  calculate the new choice probabilities with the increased price.
    ccp2 = clogit.choice_prob(thetahat, x2)
    
    # D. percentage change in CCPs 
    rel_change_y = ccp2 / ccp1 - 1.0 
    
    # E. elasticities 
    elasticity = rel_change_y / rel_change_x 
    
    E_own[:, j] = elasticity[:, j]

print(f'Own-price elasticity:  {np.mean(E_own).round(4)}')
    

Own-price elasticity:  -0.9707


Own price elasticity calculation using calculus

In [27]:
k_price = 0
dydx_J   = np.zeros((N, J))

for j in range(J):

    dydx_J[:,j]= x[:,j,k_price]*(1 - ccp1[:,j]) * thetahat[0]

    # E. elasticities 
print(f'Own price elasticity {np.mean(dydx_J).round(4)}')
# Formula from Green (Econometric Analysis)

Own price elasticity -0.9712


In [28]:
home = 1
# Create two indexed, from where idx1 is for domestic cars
# and idx0 is for imported cars.
idx1 = x[:, :, home]==1
idx0 = x[:, :, home]==0 
print(f'Elasticity, Domestic cars:   {np.mean(E_own[idx1]).round(4)}')
print(f'Elasticity, Imported cars: {np.mean(E_own[idx0]).round(4)}')

Elasticity, Domestic cars:   -1.0088
Elasticity, Imported cars: -0.953


# Numerical partial effect of Home

In [29]:
k_home = 1
dydx_J   = np.zeros((N, J))

for j in range(J):
    # A. copy 
    x3 = x.copy()
    
    # B. Set car J to home 
    x3[:, j, k_home] = 1
    # C. evaluate CCPs.  calculate the new choice probabilities with the increased price.
    ccp3 = clogit.choice_prob(thetahat, x3)
    
    dydx_J[:,j]= ccp3[:,j] - ccp1[:,j]

    # E. elasticities 
print(f'Average partial effect  {np.mean(dydx_J).round(4)}')

# Ad hoc partial effect
# Effect of "treating" all cars such that they are from "home".
# Note that some cars already are home in some markets.
# This reduce the overall effect in comparison to the next estimate.
home = 1
# Create two indexed, from where idx1 is for domestic cars
# and idx0 is for imported cars.
idx1 = x[:, :, home]==1
idx0 = x[:, :, home]==0 
print(f'Partial effect on untreated (domestic cars):   {np.mean(dydx_J[idx1]).round(4)}')
print(f'Partial effect on treated (Imported cars): {np.mean(dydx_J[idx0]).round(4)}')

Average partial effect  0.0297
Partial effect on untreated (domestic cars):   0.0
Partial effect on treated (Imported cars): 0.0435


In [30]:

k_home = 1
PE_J   = np.zeros((N, J))

for j in range(J):
 
    PE_J[:,j]= ccp1[:,j]*(1 - ccp1[:,j]) * thetahat[1]

    # E. elasticities 
print(f'Partial effect  {np.mean(PE_J).round(4)}')
# Formula from Greene (Econometric analysis)
# Does not account for cars already being from home

Partial effect  0.0325


In [31]:

k_price = 0
PE_J   = np.zeros((N, J))

for j in range(J):
 
    PE_J[:,j]= ccp1[:,j]*(1 - ccp1[:,j]) * thetahat[k_price]
    Pij = ccp1[:,j]
    # E. elasticities 
print(f'Partial effect  {np.mean(PE_J).round(4)}')
# Formula from Greene (Econometric analysis)
# Does not account for cars already being from home



print(f'Own price elasticity  {np.mean(PE_J).round(4) * (np.mean(cars["princ"])) / np.mean(Pij)}')


Partial effect  -0.0317
Own price elasticity  -1.2192364604298498


In [32]:
k_price = 0
dydx_J   = np.zeros((N, J))

for m in range(J):
    for j in range(J):
        dydx_J[:,m]= x[:,m,k_price] * (1 * (m==j) - ccp1[:,m]) * thetahat[k_price]

    # E. elasticities 
#print(f'Own price elasticity {np.mean(dydx_J).round(4)}')

own_price  = np.zeros((N, J))
for j in range(J):
    own_price[:,j] = dydx_J[:,j]

print(f'Own price elasticity {np.mean(own_price).round(4)}')
# Something is probably wrong here.

Own price elasticity -0.0054


# Calculate CI for partial effects using delta method

In [33]:
def get_se(grad, cov):
    cov_me = grad @ cov @ grad.T
    return np.sqrt(np.diag(cov_me))

se_pe_home = get_se(np.mean(PE_J,axis=0) , res['cov'][:,1])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 38 is different from 40)

In [ ]:
# print results in a nice table 
me_dict = {'Marginal Effect': np.vstack([me_home])[:,0],
           's.e.':            np.vstack([se_pe_home[3]])[:,0]}
tab = pd.DataFrame(me_dict,index=['Home'])
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(4)

# CFH attempt

In [ ]:
x_vars

In [34]:
#create mean variables with shape to match all alernatives
m_logp = np.mean(x[:, :, 0], axis=0)
m_cy = np.mean(x[:, :, 2], axis=0)
m_hp = np.mean(x[:, :, 3], axis=0)
m_we = np.mean(x[:, :, 4], axis=0)
m_li = np.mean(x[:, :, 5], axis=0)

In [35]:
m_logp.shape

(40,)

In [36]:
len(x_vars_dummies)

32

In [59]:
#construct x0
x0 = np.zeros((1, 40, len(x_vars)))
x0[0, :, 0] = m_logp
x0[0, :, 2] = m_cy
x0[0, :, 3] = m_hp
x0[0, :, 4] = m_we
x0[0, :, 5] = m_li

#remember now the brand is bmw since all brand dummies included are set to 0
#think about how to deal with this

In [56]:
#construct x1
x1 = x0.copy()
x1[0, :, 1] = 1

In [57]:
#calculate choice probabilities
prob_x0 = clogit.choice_prob(thetahat, x0)
prob_x1 = clogit.choice_prob(thetahat, x1)

#calculate the marginal effect
PE_J_new = prob_x1 - prob_x0

In [58]:
#intermezzo: calculate the average marginal effect
#calculate the average marginal effect
PE_J_new_avg = np.mean(PE_J_new, axis=0)
print(f'Average partial effect  {np.mean(PE_J_new_avg).round(4)}')

Average partial effect  -0.0


In [ ]:
#Print shapes
print(prob_x0.shape)
print(x1.shape)

In [45]:
#calculate the gradient
grad_new = prob_x1.T@(1-prob_x1)@x1 - prob_x0.T@(1-prob_x0)@x0
grad_new = grad_new.reshape((J,K))
print(grad_new.shape)

(40, 38)


In [41]:
#read the covariance matrix
cov_new = res['cov']
print(cov_new.shape)

(38, 38)


In [46]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_choice = get_se(grad_new, cov_new)